In [ ]:
# Obtener data de la Wiki de GI (esperando no ser baneado)
# Este proceso solo se debe de ejecutar 1 sola vez
# El proceso obtendrá los datos de todos los personajes y almacenará individualmente los archivos en la carpeta "datasets"

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import os

In [3]:
BASE_WIKI_URL = "https://genshin-impact.fandom.com/wiki"
OUTPUT_DIR = "en_datasets"

In [4]:
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [5]:
wmp = requests.get(BASE_WIKI_URL + "/Genshin_Impact_Wiki")

if wmp.status_code != 200:
    print("Ha ocurido un error al obtener los datos de la wiki")
    exit(-1)

In [32]:
soup = bs(wmp.text, "html.parser")

In [41]:
dicts_of_chars = {}
for elem in soup.find_all("div", { "class": "card-container"}):
    #print(elem.find("a").get("href"))
    link_tag = elem.find("a")
    pj = link_tag.get("title")
    link = link_tag.get("href").split("/")[-1]
    dicts_of_chars[pj] = "/" + link + "/Lore"

In [ ]:
dicts_of_chars

In [43]:
#DESCRIPTION_ID = "mw-content-text"
DESCRIPTION_CLASS = "mw-parser-output"

In [ ]:
from time import sleep
total_keys = len(dicts_of_chars)
for idx, k in enumerate(dicts_of_chars):
    #if idx > 5: break
    pj_wiki = requests.get(BASE_WIKI_URL + dicts_of_chars[k])
    if pj_wiki.status_code != 200:
        print("Ha ocurrido un error al obtener los datos del PJ: {}".format(k))
        sleep(5)
        continue
    
    soup = bs(pj_wiki.text, "html.parser")
    file_name = k.replace(" ", "_")
    pj_desc_container = soup.find("div", { "class": DESCRIPTION_CLASS })
    tg_childs = pj_desc_container.children
    tg_childs = [*tg_childs]
    tg_childs = [*tg_childs][6:]
    with open(OUTPUT_DIR + "/" + file_name + ".txt", "w") as f:
        f.write(k.upper() + "\n\n")
        for c in tg_childs:
            if c.name in ["p", "h2", "div" ] and not c.get("class"):
                text_content = c.text.replace("[", "").replace("]", "").strip()
                if text_content == "Namecard":
                    break
                f.write(text_content + "\n")
    print("Finish #{} of #{} -- {}".format(idx, total_keys, k))
    sleep(5)